In [1]:
# Glenda and Chris' Awesome Code!

In [2]:
#added by glenda

In [4]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
from scipy.stats import linregress

# Import API key
from api_keys import weather_api_key

# Incorporated citipy to determine city based on latitude and longitude
from citipy import citipy

Store CSV files into dataframes

In [5]:
import pandas as pd
csv_file = "Resources/atlantic.csv"
atlantic_data_df = pd.read_csv(csv_file)
atlantic_data_df.head()

,ID,Name,Date,Time,Event,Status,Latitude,Longitude,Maximum Wind,Minimum Pressure,...,Low Wind SW,Low Wind NW,Moderate Wind NE,Moderate Wind SE,Moderate Wind SW,Moderate Wind NW,High Wind NE,High Wind SE,High Wind SW,High Wind NW
0,AL011851,UNNAMED,18510625,0,,HU,28.0N,94.8W,80,-999,...,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999
1,AL011851,UNNAMED,18510625,600,,HU,28.0N,95.4W,80,-999,...,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999
2,AL011851,UNNAMED,18510625,1200,,HU,28.0N,96.0W,80,-999,...,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999
3,AL011851,UNNAMED,18510625,1800,,HU,28.1N,96.5W,80,-999,...,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999
4,AL011851,UNNAMED,18510625,2100,L,HU,28.2N,96.8W,80,-999,...,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999


In [6]:
new_atlantic_data_df = atlantic_data_df[["ID","Name","Date","Time","Status","Latitude","Longitude","Maximum Wind","Minimum Pressure"]]
new_atlantic_data_df

,ID,Name,Date,Time,Status,Latitude,Longitude,Maximum Wind,Minimum Pressure
0,AL011851,UNNAMED,18510625,0,HU,28.0N,94.8W,80,-999
1,AL011851,UNNAMED,18510625,600,HU,28.0N,95.4W,80,-999
2,AL011851,UNNAMED,18510625,1200,HU,28.0N,96.0W,80,-999
3,AL011851,UNNAMED,18510625,1800,HU,28.1N,96.5W,80,-999
4,AL011851,UNNAMED,18510625,2100,HU,28.2N,96.8W,80,-999
...,...,...,...,...,...,...,...,...,...
49100,AL122015,KATE,20151112,1200,EX,41.3N,50.4W,55,981
49101,AL122015,KATE,20151112,1800,EX,41.9N,49.9W,55,983
49102,AL122015,KATE,20151113,0,EX,41.5N,49.2W,50,985
49103,AL122015,KATE,20151113,600,EX,40.8N,47.5W,45,985


In [7]:
csv_file = "Resources/pacific.csv"
pacific_data_df = pd.read_csv(csv_file)
pacific_data_df.head()

,ID,Name,Date,Time,Event,Status,Latitude,Longitude,Maximum Wind,Minimum Pressure,...,Low Wind SW,Low Wind NW,Moderate Wind NE,Moderate Wind SE,Moderate Wind SW,Moderate Wind NW,High Wind NE,High Wind SE,High Wind SW,High Wind NW
0,EP011949,UNNAMED,19490611,0,,TS,20.2N,106.3W,45,-999,...,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999
1,EP011949,UNNAMED,19490611,600,,TS,20.2N,106.4W,45,-999,...,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999
2,EP011949,UNNAMED,19490611,1200,,TS,20.2N,106.7W,45,-999,...,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999
3,EP011949,UNNAMED,19490611,1800,,TS,20.3N,107.7W,45,-999,...,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999
4,EP011949,UNNAMED,19490612,0,,TS,20.4N,108.6W,45,-999,...,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999


In [8]:
new_pacific_data_df = pacific_data_df[["ID","Name","Date","Time","Status","Latitude","Longitude","Maximum Wind","Minimum Pressure"]]
new_pacific_data_df

,ID,Name,Date,Time,Status,Latitude,Longitude,Maximum Wind,Minimum Pressure
0,EP011949,UNNAMED,19490611,0,TS,20.2N,106.3W,45,-999
1,EP011949,UNNAMED,19490611,600,TS,20.2N,106.4W,45,-999
2,EP011949,UNNAMED,19490611,1200,TS,20.2N,106.7W,45,-999
3,EP011949,UNNAMED,19490611,1800,TS,20.3N,107.7W,45,-999
4,EP011949,UNNAMED,19490612,0,TS,20.4N,108.6W,45,-999
...,...,...,...,...,...,...,...,...,...
26132,EP222015,SANDRA,20151128,1200,LO,21.7N,109.0W,35,1002
26133,EP222015,SANDRA,20151128,1800,LO,22.4N,108.7W,30,1007
26134,EP222015,SANDRA,20151129,0,LO,23.1N,108.3W,30,1008
26135,EP222015,SANDRA,20151129,600,LO,23.5N,107.9W,25,1009


In [9]:
# combine atlantic and pacific data into a single dataframe
atlantic_and_pacific_data = pd.concat([new_atlantic_data_df, new_pacific_data_df])
atlantic_and_pacific_data

,ID,Name,Date,Time,Status,Latitude,Longitude,Maximum Wind,Minimum Pressure
0,AL011851,UNNAMED,18510625,0,HU,28.0N,94.8W,80,-999
1,AL011851,UNNAMED,18510625,600,HU,28.0N,95.4W,80,-999
2,AL011851,UNNAMED,18510625,1200,HU,28.0N,96.0W,80,-999
3,AL011851,UNNAMED,18510625,1800,HU,28.1N,96.5W,80,-999
4,AL011851,UNNAMED,18510625,2100,HU,28.2N,96.8W,80,-999
...,...,...,...,...,...,...,...,...,...
26132,EP222015,SANDRA,20151128,1200,LO,21.7N,109.0W,35,1002
26133,EP222015,SANDRA,20151128,1800,LO,22.4N,108.7W,30,1007
26134,EP222015,SANDRA,20151129,0,LO,23.1N,108.3W,30,1008
26135,EP222015,SANDRA,20151129,600,LO,23.5N,107.9W,25,1009


Data Transformation

In [10]:
atlantic_and_pacific_data.dtypes

ID                  object
Name                object
Date                 int64
Time                 int64
Status              object
Latitude            object
Longitude           object
Maximum Wind         int64
Minimum Pressure     int64
dtype: object

In [16]:
# filter data from the CSV files to shrink the data to process
# Date >= year 2000
atlantic_and_pacific_2000 = atlantic_and_pacific_data.loc[atlantic_and_pacific_data["Date"] >= 20000000, :]
# Maximum Wind >= 100
clean_atlantic_and_pacific_df = atlantic_and_pacific_2000.loc[atlantic_and_pacific_2000["Maximum Wind"] >= 100, :]
clean_atlantic_and_pacific_df

,ID,Name,Date,Time,Status,Latitude,Longitude,Maximum Wind,Minimum Pressure
41204,AL032000,ALBERTO,20000812,600,HU,35.1N,56.7W,100,960
41205,AL032000,ALBERTO,20000812,1200,HU,35.9N,55.3W,110,950
41206,AL032000,ALBERTO,20000812,1800,HU,36.8N,53.8W,110,954
41207,AL032000,ALBERTO,20000813,0,HU,37.4N,52.0W,105,958
41437,AL132000,ISAAC,20000924,0,HU,14.9N,35.0W,105,960
...,...,...,...,...,...,...,...,...,...
26122,EP222015,SANDRA,20151126,0,HU,13.3N,110.2W,110,957
26123,EP222015,SANDRA,20151126,600,HU,14.1N,110.2W,130,934
26124,EP222015,SANDRA,20151126,1200,HU,15.2N,110.2W,120,943
26125,EP222015,SANDRA,20151126,1800,HU,16.3N,110.2W,115,947


In [30]:
# convert latitude and longitude values to decimal format
def cardinal2negative(s):
    degrees = float(s[:-1])
    cardinal = s[len(s)-1:]
    
    if cardinal in ('s','S','w','W'):
        degrees *= -1

    return degrees

#data = [['35.1N', '56.7W'], ['35.9N', '107.2E'], ['55.0S', '13.1W']] 
## Create the pandas DataFrame 
#df = pd.DataFrame(data, columns = ['Latitude', 'Longitude']) 
#df['Latitude'].apply(cardinal2negative)
#df['Longitude'].apply(cardinal2negative)

clean_atlantic_and_pacific_df['New Latitude'] = clean_atlantic_and_pacific_df['Latitude'].apply(cardinal2negative)
clean_atlantic_and_pacific_df['New Longitude'] = clean_atlantic_and_pacific_df['Longitude'].apply(cardinal2negative)
clean_atlantic_and_pacific_df


C:\Users\sadlo\Anaconda3\envs\NewPythonData\lib\site-packages\ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\sadlo\Anaconda3\envs\NewPythonData\lib\site-packages\ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,ID,Name,Date,Time,Status,Latitude,Longitude,Maximum Wind,Minimum Pressure,New Latitude,New Longitude
41204,AL032000,ALBERTO,20000812,600,HU,35.1N,56.7W,100,960,35.1,-56.7
41205,AL032000,ALBERTO,20000812,1200,HU,35.9N,55.3W,110,950,35.9,-55.3
41206,AL032000,ALBERTO,20000812,1800,HU,36.8N,53.8W,110,954,36.8,-53.8
41207,AL032000,ALBERTO,20000813,0,HU,37.4N,52.0W,105,958,37.4,-52.0
41437,AL132000,ISAAC,20000924,0,HU,14.9N,35.0W,105,960,14.9,-35.0
...,...,...,...,...,...,...,...,...,...,...,...
26122,EP222015,SANDRA,20151126,0,HU,13.3N,110.2W,110,957,13.3,-110.2
26123,EP222015,SANDRA,20151126,600,HU,14.1N,110.2W,130,934,14.1,-110.2
26124,EP222015,SANDRA,20151126,1200,HU,15.2N,110.2W,120,943,15.2,-110.2
26125,EP222015,SANDRA,20151126,1800,HU,16.3N,110.2W,115,947,16.3,-110.2


In [39]:
# Determine the nearest city for each lat lng pair and add it to the dataframe
unique_cities_list = []
city_list = []
country_list = []

# Create a set of random lat and lng combinations
lats = clean_atlantic_and_pacific_df["New Latitude"]
lngs = clean_atlantic_and_pacific_df["New Longitude"]
lat_lngs = zip(lats, lngs)

# Identify nearest city for each lat, lng combination
for lat_lng in lat_lngs:
    city = citipy.nearest_city(lat_lng[0], lat_lng[1]).city_name
    country = citipy.nearest_city(lat_lng[0], lat_lng[1]).country_code
    
    city_list.append(city)
    country_list.append(country)
    
    city_country_pair = f"{city}, {country}"
    
    # If the city is unique, then add it to a our cities list
    if city_country_pair not in unique_cities_list:
        unique_cities_list.append(city_country_pair)

# Add the city and country pair to the dataframe here:
clean_atlantic_and_pacific_df["City"] = city_list
clean_atlantic_and_pacific_df["Country"] = country_list
        
# Print the city count to confirm sufficient count
len(unique_cities_list)
unique_cities_list

C:\Users\sadlo\Anaconda3\envs\NewPythonData\lib\site-packages\ipykernel_launcher.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\sadlo\Anaconda3\envs\NewPythonData\lib\site-packages\ipykernel_launcher.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


['saint george, bm',
 'saint-pierre, pm',
 'porto novo, cv',
 'ponta do sol, cv',
 'codrington, ag',
 'san pedro, bz',
 'barra patuca, hn',
 'savannah bight, hn',
 'dangriga, bz',
 'punta gorda, bz',
 'west bay, ky',
 'santa fe, cu',
 'jaguey grande, cu',
 'aguada de pasajeros, cu',
 'isla mujeres, mx',
 'kantunilkin, mx',
 'panaba, mx',
 'dzilam gonzalez, mx',
 'dzemul, mx',
 'tixkokob, mx',
 'progreso, mx',
 'houma, us',
 'morgan city, us',
 'bathsheba, bb',
 'saint-francois, gp',
 'the valley, ai',
 'road town, vg',
 'hamilton, bm',
 'cayenne, gf',
 'sinnamary, gf',
 'san juan, us',
 'cockburn town, tc',
 'cockburn town, bs',
 'nantucket, us',
 'shelburne, ca',
 'port hawkesbury, ca',
 'alquizar, cu',
 'bauta, cu',
 'fort myers beach, us',
 'iona, us',
 'punta gorda, us',
 'dorado, us',
 'samana, do',
 'cockburn harbour, tc',
 'rock sound, bs',
 'mana, gf',
 'scarborough, tt',
 'grenville, gd',
 'gouyave, gd',
 'la asuncion, ve',
 'kralendijk, an',
 'rincon, an',
 'oranjestad, aw',


In [40]:
clean_atlantic_and_pacific_df.head()

,ID,Name,Date,Time,Status,Latitude,Longitude,Maximum Wind,Minimum Pressure,New Latitude,New Longitude,City,Country
41204,AL032000,ALBERTO,20000812,600,HU,35.1N,56.7W,100,960,35.1,-56.7,saint george,bm
41205,AL032000,ALBERTO,20000812,1200,HU,35.9N,55.3W,110,950,35.9,-55.3,saint george,bm
41206,AL032000,ALBERTO,20000812,1800,HU,36.8N,53.8W,110,954,36.8,-53.8,saint-pierre,pm
41207,AL032000,ALBERTO,20000813,0,HU,37.4N,52.0W,105,958,37.4,-52.0,saint-pierre,pm
41437,AL132000,ISAAC,20000924,0,HU,14.9N,35.0W,105,960,14.9,-35.0,porto novo,cv


In [42]:
clean_atlantic_and_pacific_df.shape

(967, 13)